In [ ]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import arrow_dataset, storage_adaptor
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from workflows import sampler_task, sampler_flow
from utilities import date_fu, project_globals as g
from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
prefix_data = '/data/trades'

all_syms = data_access.list_sd_data(prefix=prefix_data)

config = {
    'meta': {
        'symbol': 'EGO',
        'symbol_list': all_syms,
        'start_date': '2019-01-01',
        'end_date': '2020-01-01',
        'config_id': 'renko_v1',
        'presist_destination': 'both',
        'ray_on': True,
    },
    'filter': {
        'mad_value_winlen': 22,
        'mad_deviation_winlen': 1111,
        'mad_k': 17,
        'jma_winlen': 7,
        'jma_power': 2,
    },
    'sampler': {
        'renko_return': 'price_jma_return',
        'renko_size': 0.1,  # for simple runs
        'renko_reveral_multiple': 2,
        'renko_range_frac': 22,
        'renko_range_min_pct_value': 0.04,  # % of symbol value enforced as min renko size
        'max_duration_td': dt.timedelta(minutes=33),
        'min_duration_td': dt.timedelta(seconds=33),
        'min_tick_count': 33,
        'add_label': True,
        'reward_ratios': list(np.arange(2, 11, 0.5)),
    }
}

prefix_2 = f"/bars/{config['meta']['config_id']}/meta"

prefix_3 = f"/bars/{config['meta']['config_id']}/df"

In [9]:
data_access.list_sd_data(prefix=prefix_2, source='local', show_storage=True)

{'MB': 135.7}

# Read flow

In [ ]:
lp = data_access.walk_prefix(prefix_2, source='local')

bdf = read_flow.run(lp, data_access.fs_local.read_df_from_fs, return_df=True)

In [ ]:
ds = arrow_dataset.get_dataset(prefix_2, fs_type='local')

df = ds.to_table().to_pandas()

In [ ]:
df.groupby(['symbol','date']).mean().reset_index().groupby('symbol').mean().describe()

In [ ]:
df.groupby(['symbol','date']).count().reset_index().groupby('symbol').mean().describe()

# Tick Sampler 

In [10]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

2021-06-04 22:48:29,707	INFO services.py:1267 -- View the Ray dashboard at http://0.0.0.0:1111


{'node_ip_address': '192.168.1.66',
 'raylet_ip_address': '192.168.1.66',
 'redis_address': '192.168.1.66:6379',
 'object_store_address': '/tmp/ray/session_2021-06-04_22-48-27_260600_25122/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-04_22-48-27_260600_25122/sockets/raylet',
 'webui_url': '0.0.0.0:1111',
 'session_dir': '/tmp/ray/session_2021-06-04_22-48-27_260600_25122',
 'metrics_export_port': 65499,
 'node_id': '2742754521bc710dabb309690bf1787de359db9002899421091af7fa'}

In [3]:
ray.shutdown()

In [ ]:
bds = sampler_flow.run(config)

No remaining dates
failed symbol list: AEM
No remaining dates
failed symbol list: AG
No remaining dates
failed symbol list: AGI
No remaining dates
failed symbol list: ASA
No remaining dates
failed symbol list: AU
No remaining dates
failed symbol list: AUY
No remaining dates
failed symbol list: AXU
No remaining dates
failed symbol list: BTG
No remaining dates
failed symbol list: BVN
No remaining dates
failed symbol list: CDE
No remaining dates
failed symbol list: DRD
No remaining dates
failed symbol list: EGO
No remaining dates
failed symbol list: EXK
No remaining dates
failed symbol list: FNV
No remaining dates
failed symbol list: FSM
No remaining dates
failed symbol list: GFI
No remaining dates
failed symbol list: GLD
(pid=25170) running GORO 2019-05-09
(pid=25168) running GORO 2019-05-10
(pid=25171) running GORO 2019-05-07
(pid=25169) running GORO 2019-05-08
(pid=25170) running GORO 2019-05-13
(pid=25171) running GORO 2019-05-14
(pid=25168) running GORO 2019-05-15
(pid=25169) running

In [ ]:
data_access.fs_remote.write_sd_data(out[0], 'test', '2020-01-01', prefix_2)

In [ ]:
n = 1
print(bds[n]['date'])
# bds[n]['ticks_df'][['price', 'price_jma']].plot_bokeh(sizing_mode="scale_height")
bds[n]['ticks_df'][['price', 'price_jma']].plot()

In [ ]:
bds[n]['ticks_df'][['price_jma']].plot()

In [ ]:
bds[n]['ticks_df'].status.value_counts() / len(bds[n]['ticks_df'])

In [ ]:
bds[n]['bars_df'][['price_low', 'price_high', 'price_close']].plot()

# Price gap fill

In [ ]:
from sample_features import stacked
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bar_dates, fill_col='price_vwap')

# stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)


In [ ]:
stats_df.describe()